# Homework 4 

## To be submitted by Thursday November 10, 2016

In this homework, we will look at some basic "machine learning", using linear algebra and optimization.

The simplest type of classification problem is the following:

Given a large number $M$ of data samples $(\mathbf{x}_i)_{i=1}^M \in \mathbb{R}^n$ (the so-called "training set"), and known labels $y_i \in Y$ for each of them, coming from a finite set $Y$, we want to discover ("learn") a function 

$$f_\mathbf{\mu} : \mathbb{R}^n \to Y$$

that classifies the data. We restrict the form of the function $f$ to a certain class, and parameterize it by a set of $p$ parameters $\mathbf{\mu} = (\mu_i)_{i=1}^p$. The problem then reduces to finding the "best" (optimal) parameters $\mathbf{\mu}$, to minimise a "cost function" $C$. This cost function is expressed in terms of the distance from the prediction $f(x_i)$ to the true $y_i$, and the cost function $C$ is some kind of agregate of these distances, for example their sum over all the data $i$.  

That is, given:
1. the data $\mathbf{x}_i$;
2. the corresponding labels $y_i$;
3. the functional form of $f$ containing some unknown parameters; and
4. a cost function $C$, that is a scalar function of all the $\mathbf{x}_i$s and $y_i$s.

we wish to find the values of the parameters in $f$ that (approximately) minimise the cost function.

Since finding the right parameters is usually an iterative process, given by an optimization algorithm, that is supposed to converge (in some suitable sense) to the best solution, this can be thought of as a "learning" process.

The quality of the resulting prediction function is then checked by testing it on a separate set of data (the "test set").

We will, however, begin with non-iterative algorithms that are simpler to implement, but have the disadvantage that all data must be known at once.

## 0. Loading the data

The data we will use is a simplified and reduced version of the [MNIST](http://yann.lecun.com/exdb/mnist/) set of handwritten digits, one of the most commonly-used data sets in machine learning. 

This version of the data was kindly provided by [Prof. Raj Rao](http://web.eecs.umich.edu/~rajnrao/), University of Michigan.

(i) Load the training data from [`training_digits.jld`](training_digits.jld) using the [`JLD.jl`](https://github.com/JuliaIO/JLD.jl) package, and explore it.

The data consists of a 3-dimensional array ("tensor"):

- The third component runs from 1 to 10, and specifies the digit from 0 to 9. 
- The columns $\mathbf{x}_i$ (the first component) are vectors of length $n=256$ representing grayscale images of $16 \times 16$ pixels. 
- The second component runs over the number of training examples.

In the same file are the corresponding labels $y_i \in \{0, \ldots, 9\} =: Y$.

Load the test data from [`test_digits.jld`](test_digits.jld).

(ii) Visualize some of the digits by reshaping them to a $16 \times 16$ array and plotting the resulting image, e.g. using `matshow` in PyPlot. Use e.g. `@manipulate`. Note that some of the data is all zero.

## 1. Least squares

One of the first ideas that we could have is to look for a simple function $f^{(i)}$  that classifies each digit as e.g. "being a 7 or not being a 7". We could thus look for a linear map (the simplest type of map) from $\mathbb{R}^n$ to $\mathbb{R}$ that maps $\mathbf{x}_i$ to a $1$ or $0$, depending if the image does or does not correspond to a digit of type $i$.

(iii) Express the "being a 7" problem as a matrix-vector multiplication over a single matrix containing all the data. Use least squares (`\`) to solve the problem.

(iv) Stacking up all such problems vertically gives a problem where the vector $\mathbf{x}_i$ is mapped onto a **one-hot vector** $\mathbf{y}_i$ representing the digit $y_i$. It turns out that some of the data vectors are all zero. Remove these and make one single matrix $\mathsf{x}$ by horizontally stacking the data, and another, $\mathsf{y}$ the corresponding one-hot vectors. 

Using least squares, find the matrix $\mathsf{W}$ that best solves $\mathsf{W} \mathbf{x}_i \approx \mathbf{y}_i$ for each $i$. (NB: Take care about the dimensions. What are you solving for?) 

Use the resulting matrix $\mathsf{W}$ to classify the test digits. What proportion does it get right?

(v) Instead of just $\mathsf{W} \mathbf{x}_i = \mathbf{y}_i$, do the same with $\mathsf{W} \mathbf{x} + \mathbf{b} = \mathbf{y}_i$, by adding an extra $1$ at the bottom of each $\mathbf{x}_i$.

(vi) Use `@manipulate` to scroll through misclassified images and discuss their features.

## 2. SVD

(vii) To get a feel for the SVD, decompose your favorite image using it, following the instructions in the [SVD for images](SVD for images.ipynb) notebook.

An approach to the learning problem via the SVD is as follows.

(viii) For each set of training data (i.e. for the data corresponding to a given digit), take the SVD of the data and use it to approximate the data by a rank-$k$ hyperplane through the origin, i.e. to find the closest hyperplane of dimension $k$ passing through the data.

Classify the test digits by finding to which of these hyperplanes it is closest, using a suitable **projection matrix**.
How well does this classification method do on the test data? How does this depend on $k$?

**Optional: Does it improve the classification if you use hyperplanes that do not go through the origin?

## 3. Stochastic gradient descent

Now we will repeat the least-squares calculation, but using a technique that generalizes to (much) more complicated scenarios: **stochastic gradient descent**.

We wish to minimise the least-squares cost function $C := \sum_i C_i$, where $C_i(x) := \|\mathsf{W} \mathbf{x}_i - \mathbf{y}_i \|^2$, using gradient descent. However, the function depends on many variables (all the $\mathbf{x}_i$), so calculating its gradient is very expensive.

In stochastic gradient descent, we instead choose a single, random $i$ and update the matrix $\mathsf{W}$ by using gradient descent with respect to $C_i$ only, where we think of $C_i$ as a function of $\mathsf{W}$. 

The difficult part about using gradient descent is usually calculating and evaluating these gradients; in general, we use reverse automatic differentiation (called **backpropagation** in the neural networks community). However, in this particular example, the gradient is simple enough to calculate by hand.

(ix) Show that the gradient of $f(\mathbf{x}) := \| \mathsf{W} \mathbf{x} - \mathbf{y} \|^2$ with respect to the matrix $\mathsf{W}$ is $2 \, (\mathsf{W} \mathbf{x} - \mathbf{y}) \, \mathbf{x}^\mathrm{T}$. 

(x) Use this to do stochastic gradient descent with a step size of $\eta = 0.001$, choosing one $i$ randomly at each step and moving in the direction that minimises $C_i$. 

After each 1000 steps, calculate the proportion of test cases that are correctly classified and draw this as a function of number of steps. How many steps are required to get close to the optimal solution, given by the linear algebra result in section 1? How well does the classification do compared to that?


**Optional: Use `ForwardDiff` to calculate the gradient. Note that you will need to be careful about when to treat $\mathsf{W}$ as a matrix, and when to treat it as a vector. Compare this to reverse automatic differentiation from the new https://github.com/JuliaDiff/ReverseDiff.jl package (once the documentation is available!). Which is faster?